In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/climate-change-edsa2020-21/train.csv
/kaggle/input/climate-change-edsa2020-21/test.csv
/kaggle/input/climate-change-edsa2020-21/sample_submission.csv


Load Libraries

In [2]:
# import libraries

import numpy as np
import pandas as pd
import re
import nltk
from sklearn.datasets import load_files

import string
from nltk.corpus import stopwords
from sklearn.metrics import f1_score
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import LinearSVC


Read the datasets

In [3]:

test = pd.read_csv('../input/climate-change-edsa2020-21/test.csv')
train = pd.read_csv('../input/climate-change-edsa2020-21/train.csv')


In [4]:
test.head()

,message,tweetid
0,Europe will now be looking to China to make su...,169760
1,Combine this with the polling of staffers re c...,35326
2,"The scary, unimpeachable evidence that climate...",224985
3,@Karoli @morgfair @OsborneInk @dailykos \nPuti...,476263
4,RT @FakeWillMoore: 'Female orgasms cause globa...,872928


In [5]:
train.head()


,sentiment,message,tweetid
0,1,PolySciMajor EPA chief doesn't think carbon di...,625221
1,1,It's not like we lack evidence of anthropogeni...,126103
2,2,RT @RawStory: Researchers say we have three ye...,698562
3,1,#TodayinMaker# WIRED : 2016 was a pivotal year...,573736
4,1,"RT @SoyNovioDeTodas: It's 2016, and a racist, ...",466954


Split data into X and y

In [6]:

y = train['sentiment']
X = train['message']

In [7]:
from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

def my_preprocessor(text):
    
      # split into words
    from nltk.tokenize import word_tokenize
    tokens = word_tokenize(text)
    # convert to lower case
    tokens = [w.lower() for w in tokens]
    # remove punctuation from each word
    import string
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]
   
    # filter out stop words
    from nltk.corpus import stopwords
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
       
    # lemmatize
    lemmatized_words = [stemmer.lemmatize(word) for word in words]
    return ' '.join(lemmatized_words)

In [8]:
def my_tokenizer(text):
    # create a space between special characters 
    text=re.sub("(\\W)"," \\1 ",text)

    # split based on whitespace
    return re.split("\\s+",text)


Vectorise the data so the model can undertands the data

In [9]:
# vectorise the data so the model can undertands
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfconverter  = TfidfVectorizer(ngram_range=(1,2),tokenizer=my_tokenizer, min_df=2,max_df=0.80,analyzer='word',smooth_idf=False, preprocessor=my_preprocessor,stop_words="english")
X_vectorized    = tfidfconverter .fit_transform(X)


/opt/conda/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['', 'le', 'u'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [10]:
count_vect_df = pd.DataFrame(X_vectorized)
count_vect_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15819 entries, 0 to 15818
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       15819 non-null  object
dtypes: object(1)
memory usage: 123.7+ KB


Split the data into Test and Train

In [11]:
#split the data into Test and Train

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_vectorized,y,test_size=0.20,shuffle=True, random_state=25)

In [12]:
Train the model

SyntaxError: invalid syntax (<ipython-input-12-3338c251285d>, line 1)

In [13]:
# To train our machine learning model using the LinearSVC 
lsvc = LinearSVC()
lsvc.fit(X_train, y_train)

LinearSVC()

In [14]:
lsvc_pred = lsvc.predict(X_test)

Check Model Accuracy

In [15]:
# get f1_score
f1_score=f1_score(y_test, lsvc_pred, average="macro")


In [16]:
print(f1_score)

0.6538674206591586


In [17]:
from sklearn import metrics

print(metrics.classification_report(y_test, lsvc_pred))

              precision    recall  f1-score   support

          -1       0.74      0.42      0.53       247
           0       0.58      0.47      0.52       466
           1       0.77      0.86      0.81      1749
           2       0.76      0.74      0.75       702

    accuracy                           0.74      3164
   macro avg       0.71      0.62      0.65      3164
weighted avg       0.74      0.74      0.73      3164



In [18]:
# model with test data
y_pred = lsvc.predict( tfidfconverter.transform(test['message']))

In [19]:
#create sentiment from the predicts
test['sentiment'] = y_pred
test.head()

,message,tweetid,sentiment
0,Europe will now be looking to China to make su...,169760,1
1,Combine this with the polling of staffers re c...,35326,1
2,"The scary, unimpeachable evidence that climate...",224985,1
3,@Karoli @morgfair @OsborneInk @dailykos \nPuti...,476263,1
4,RT @FakeWillMoore: 'Female orgasms cause globa...,872928,1


Convert output to csv

In [20]:
# convert output to csv
test[['tweetid','sentiment']].to_csv('tebogo_mokgonyana_Log_regresssion_submission_3.csv', index=False)